<a target="_blank" href="https://colab.research.google.com/github/AndreasRupp/ecdf_estimator_examples/blob/bsc_thesis/tutorial/04-deviation-and-mean-of-normal-distribution.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Parameter Identification for Normal Distribution Data - Standard and Bootstrap Methods: Estimating Mean and Deviation

In the first notebook of the tutorial, only the deviation parameter of normal distribution data was estimated using standard method. Here, both deviation and mean parameter in normal distribution data are identified. This notebook enables using of standard or bootstrap method. The beginning of the notebook is quite similar to the first ones, and this part is covered with less detail in this notebook. Therefore, it is recommended to read the previous notebooks before proceeding further in this one. The main differences are located in the 'Evaluation of Parameters' section, where different parameter combinations are evaluated. The approach used in the identification is based on the empirical cumulative distribution function (eCDF). First, normal distribution data is created using known deviation and mean parameters. Subsequently, new data is generated using various different parameter combinations to identify the initial parameters. The implementation of the method using Python and ecdf-estimator package is explained step-by-step within this notebook.

## Setup of the Process

###Environment Setup

First, `ecdf-estimator` is installed and imported as `ecdf`.

In [ ]:
pip install ecdf-estimator

In [ ]:
import ecdf_estimator as ecdf

Next, the other two necessary modules, `numpy` and `matplotlib.pyplot`, are imported. The modules are used to do array operations efficiently and to visualize the data and results.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### Creation of Training Data

Next, normally distributed training data must be created. First, the variables which are used to create the data are defined:

* `mean`: The training data's mean value, which is tried to identify.<br>
* `dev`: The training data's standard deviation, which is tried to identidy.<br>
* `size`: The number of created data points.<br>

In [ ]:
mean = 5
dev  = 5
size = 10000

After the parameters are defined, the normally distibuted training data can be created using them and `numpy`'s `random.normal` function.

In [ ]:
data = np.random.normal(mean, dev, size)

If desired, noise can be added to the training data by setting the variable `noise_dev` as the deviation of the noise distribution. By default, `noise_dev` is set as `None` and noise is not added.

In [ ]:
noise_dev = None

if noise_dev is not None:
  data += np.random.normal(mean, noise_dev, size)

## Parameter Identification Process

After the creation of training data, the actual parameter identification begins.

### Definition of Variables

The variables which will be used in the process are defined.

* `ecdf_type`: Defines the identification method used. By default standard method is used, but also bootstrap is possible.

In [ ]:
ecdf_type = "standard"
#ecdf_type = "bootstrap"

* `subset_sizes`: The list of subset sizes of training data. Also the sizes of new datasets, which will be created later, will depend on this variable. Note that the sum of the sizes of subsets should equal the size of training data!

In [ ]:
subset_sizes = [250]*40

* `n_bins`: The maximum number of bins which will be used for the eCDF vectors. The exact number will be defined by the `ecdf`'s own functions.

In [ ]:
n_bins = 7

* `min_dev`: The start deviation parameter to be estimated.<br>
* `max_dev`: The end deviation parameter to be estimated.<br>
* `min_mean`: The start mean parameter to be estimated.<br>
* `max_mean`: The end mean parameter to be estimated.<br>

`min_dev` and `max_dev` define the interval between which all integer values are tested to be the standard deviation of our training data. `min_mean` and `max_mean` work similarly for the mean value.


In [ ]:
min_dev = 2
max_dev = 8
min_mean = 2
max_mean = 8

Next, a function named `distance`, which has a major part in the creation of eCDF vectors in the whole identification process, must be defined. The function computes the absolute value of the subtraction of the data points, which is the so called Euclidian distance between the points.


In [ ]:
def distance(data_a, data_b):
  return np.abs(data_a-data_b)

###Generation of Objective Functions and eCDF vectors

Next, the eCDF vectors have to be created and plotted. This is done by initializing a new instance of `estimator`'s class object function. Because the vectors are wanted to plot using a large number of bins and with smaller amount of bins, they must be created twice.

The `ecdf-estimator`'s `estimate_radii_values` is called to determine appropriate region values for the bin values. The values are selected based on the computed distances between data points of the first two subsets of the training data. The returned region values are stored to `min_val` and `max_val` and the distances between data points to the matrix `distance_data`.

In [ ]:
min_val, max_val, distance_data = ecdf.estimate_radii_values(data, subset_sizes, distance)

Then, the following interval is split into 50 bins.


In [ ]:
bins = np.linspace(min_val, max_val, 50)

The objective function is assembled by initializing a new instance of `estimator`'s `standard` or `bootstrap` class object. The distances between subsets of the training data are calculated, and the eCDF vector of each subset pair is created by cumulatively calculating how many of these distances belong to a certain of the 50 bins. The eCDF vectors are formed based on the method chosen as explained in the previous notebooks. Also, the mean and covariance of all vectors are computed. All these values are stored to `aux_func` to specify the first objective function with all 50 bins.

In [ ]:
if (ecdf_type == "standard"):
  aux_func = ecdf.standard(data, bins, distance, subset_sizes)
elif (ecdf_type == "bootstrap"):
  aux_func = ecdf.bootstrap(data, bins, distance, subset_sizes[0], subset_sizes[1])
else:
  print("Invalid eCDF type.")

Then, the other function with smaller amount of bins must be defined. Otherwise there could be unwanted correlation between neighbouring bin values. The `estimator`'s `choose_bins` function is called to select the reasonable bin values from larger choice.

In [ ]:
bins = ecdf.choose_bins(distance_data, bins, n_bins)

Now, the second objective function `func` can be defined. This time, only the small amount of bins is used to create the eCDF vectors and compute the statistics for them.


In [ ]:
if (ecdf_type == "standard"):
  func = ecdf.standard(data, bins, distance, subset_sizes)
elif (ecdf_type == "bootstrap"):
  func = ecdf.bootstrap(data, bins, distance, subset_sizes[0], subset_sizes[1])
else:
  print("Invalid eCDF type.")

After the two objective functions have been created, they can be plotted. First, the figure is defined using `matplotlib.pyplot`'s `subplots` function. Then, the plots are made using the `estimator`'s functions. The vectors with 50 bins are plotted as purple, the ones with selected bins only as light blue, and their mean values as black.


In [ ]:
fig, ax = plt.subplots(figsize=(12,4))
ecdf.plot_ecdf_vectors(aux_func, ax, 'm.')
ecdf.plot_ecdf_vectors(func, ax, 'c.')
ecdf.plot_mean_vector(func, ax, 'k.')
plt.title("Distribution of the eCDF-vectors")
plt.xlabel("Bin Values")
plt.ylabel("Cumulative Probability")
plt.show()

###$\chi^2$ test: Checking normality of eCDF vectors

The eCDF vectors should be Gaussian with big enough sample size of training data. This is ensured using the $\chi^2$ test which is done by calling the `estimator`'s `plot_chi2_test` function. The function computes the negative log-likelihood values for each eCDF vector to examine how well each vector fits the objective function `func`'s model. The log-likelihood values are then normalized and a histogram of them is created and plotted. After that, the probability density function of the chi-square distribution with an appropriate degree of freedom (which is the number of the eCDF vectors) is plotted on top of the histogram. If the histogram fits the density function well, the normality of eCDF vectors is confirmed and the parameter estimation process should be valid and reliable.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
ecdf.plot_chi2_test(func, ax)
plt.title("Gaussianity Test by Chi-squared -criterion")
plt.xlabel("Normalized Log-likelihood")
plt.ylabel("Probability Density")
plt.show()

###Evaluation of Parameters

After ensuring the reliability of the identification process, the evaluation of different parameter combinations begins. First, lists containing all deviation parameters and mean parameters, are created. The numbers of parameters are stored to `n_devs` and `n_means` by computing the lengths of the lists.

In [ ]:
devs = list(range(min_dev, max_dev + 1))
means = list(range(min_mean, max_mean + 1))
n_devs = len(devs)
n_means = len(means)

Next, a matrix of lists, `values` is intialized. The first for-loop creates a new empty list `n_means` times, one for each mean parameter. Then, the other for-loop creates a new list `n_devs` times for each deviation parameter, and these lists contain the empty lists for mean parameters. The resulted matrix rows are deviation parameter values and columns are mean values. For each parameter combination there is a list which will store objective function values, so called negative log-likelihood values, for datasets created for the combination. A single value of a matrix can be selected with command `matrix[i][j][k]`, `i` being the row (deviation) index, `j` the column (mean) index, and `k` the index of a list in current position.

In [ ]:
values = [[[] for i in range(n_means)] for j in range(n_devs)]

A matrix `means_log` is initialized to store mean values of log-likelihood values for each combination. Every row of the matrix contains a list for a deviation parameter. Columns will stand for mean parameters after the mean values for combinations are added to the lists.

In [ ]:
means_log = [[] for i in range(n_devs)]

A variable `n_subsets` defines, how many new data subsets are created for each parameter combination to be evaluated with the objective function.


In [ ]:
n_subsets = 5

####Negative Log-likelihood Analysis

The creation and evaluation of new datasets starts and the negative log-likelihood values will be computed. `n_subsets` new datasets are wanted to create for each parameter combination. Thus, the two outer loops iterate `n_devs` and `n_means` times, and the inner loop for each subset `n_subsets` times. In each iteration loop, a new dataset `newdata` with the current mean parameter `means[j]` and deviation parameter `devs[i]` is created using numpy's `random.normal` function. The size of a new dataset is the same as the similarly indexed dataset's size in training data, `subset_sizes[k]`.

Then, the `estimator`'s `evaluate` function is called to evaluate the new dataset with the objective function `func`. First, one subset of the training data is selected and the distances between each point of the training data subset and the new dataset are computed. Again, an eCDF function of the distances is created, similarly to what was done before with training data subsets. Then, the negative log-likelihood value of the new eCDF vector is calculated to examine how it fits the objective function created with training data. This value is the one which the function returns, and it is stored into matrix `values` by appending it to the end of the current parameter combination's list with `values[i][j].append()`.

When the inner loop is exited, the negative log-likelihood values for all datasets with certain parameter combination are computed. Thus, the mean value of that combination is computed with `numpy`'s `mean` function and stored in matrix `means_log`. When also the middle loop is exited, all values for one deviation parameter with different mean parameters are computed and their mean values are stored to `means_log[i]`.

In [ ]:
for i in range(n_devs):
  for j in range(n_means):
    for k in range(n_subsets):
      newdata = np.random.normal(means[j], devs[i], subset_sizes[k])
      values[i][j].append(ecdf.evaluate(func, newdata))
    means_log[i].append(np.mean(values[i][j]))

The evaluated log-likelihood values for all datasets and their means are plotted. The figure and axes are defined. This time, 3 similar 3D plots are made next to each other from different angles of view. For this reason, the number of rows and columns are defined, and the 3D projection is created with `subplot_kw={'projection': '3d'}`. The variable `axs` contains the axes for all three subplots.

The plotting is done with for-loops, where the first iteration is for the three axes. Their labels are set in the beginning. Then, the second loop iterates through all parameters. The log-likelihood values (red) and their means (blue) are then plotted using `ax.scatter3D`. Its' first three arguments are the x, y and z-coordinate of the plot. After that, the colour and size of plots can be defined.

The different angles of view are made with axes' `view_init` function. The first parameter, `elev`, rotates the camera above the plane by the vertical axis. The second parameter, `azim`, rotates the camera about the vertical axis. The first angle of view is to get a general view of the values, second angle to evaluate deviation parameters, and third to evaluate mean parameters. Axes' `set_box_aspect` is used to zoom the figures that z-labels can be seen. With `plt.subplots_adjust` the space between subets is set to be appropriate. `set_xticklabels` and `set_yticklabels` are used to remove ticks from the labels which can't be seen from the views. The figure's `suptitle`-function is used to get one title for all the subplots, and its' font size is defined.
    

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=3, subplot_kw={'projection': '3d'}, figsize=(21, 6))

for ax in axs:
  ax.set_xlabel('Deviation')
  ax.set_ylabel('Mean')
  ax.set_zlabel('Neg. Log-likelihood')

  for i in range(n_devs):
    for j in range(n_means):
      ax.scatter3D(devs[i], means[j], values[i][j], c='red', s=3)
      ax.scatter3D(devs[i], means[j], means_log[i][j], c='blue', s=20)

axs[0].set_box_aspect(aspect=None, zoom=0.85)
axs[1].set_box_aspect(aspect=None, zoom=1.2)

axs[0].view_init(elev=10, azim=45)
axs[1].view_init(elev=0, azim=-90)
axs[2].view_init(elev=0, azim=0)

axs[1].set_yticklabels([])
axs[2].set_xticklabels([])

plt.subplots_adjust(wspace=0, right=0.9)

fig.suptitle("Evaluation of the Negative Log-likelihood Values", fontsize=15)
plt.show()

####Evaluating the Impact of Mean and Deviation Parameters on Log-Likelihood Values

After computing the log-likelihood values for all parameter combinations, it is also possible to evaluate how much mean and deviation parameters alone affect the log-likelihood values. For that reason, the mean values of the mean log-likelihood values for parameter combinations are computed for each mean parameter and for each deviation parameter. Basically this means that all parameter combination log-likelihood means with certain deviation parameter, for example, are selected, and the mean value of them is computed. When computed this way, with the help of `means_log`, the possibly different subset sizes don't affect the results that much.


The mean values for both parameters are computed using `np.mean` function. When using `axis` as a second argument, the mean value of matrix `means_log` is computed along the specified axis. Because the values from the matrix are selected like `means_log[i][j]`, `axis=0` refers to the axis with the index 0, `i`-axis, which include all deviation parameters. Thus, when the mean is computed along that axis, there is one value for each deviation parameter with the same mean parameter included in the computation, so actually the mean values for each mean parameter are computed! Similarly, with `axis=1`, the mean value can be computed for all deviation parameters.

In [ ]:
means_mean = np.mean(means_log, axis=0)
means_dev = np.mean(means_log, axis=1)

Next, the means of likelihood values for all combinations are plotted with red, this time on 2D figures based on only the mean parameter and only the deviation parameter. Then, the just computed means of them are plotted with blue. When plotting the means of likelihood values based on mean parameter, the `means_log` matrix is transposed. That is, because with matrix as y-coordinates, `plot` function handles each row of the matrix as the corresponding y-coordinates for the current x-coordinate. Each row of `means_log` contains values with one deviation parameter, but after transposing, rows stand for values with one mean parameter. The plots should be quite similar to the last 2 views of the 3D figure, but it might be easier to evaluate the situation with the 2D figure.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(18, 5))

ax[0].plot(means, np.transpose(means_log), 'ro')
ax[0].plot(means, means_mean, 'bo')
ax[1].plot(devs, means_log, 'ro')
ax[1].plot(devs, means_dev, 'bo')

ax[0].set_xlabel("Mean Parameter")
ax[0].set_ylabel("Log-likelihood")

ax[1].set_xlabel("Deviation Parameter")
ax[1].set_ylabel("Log-likelihood")


fig.suptitle("Impact of Mean and Deviation on Log-likelihood Values", fontsize=15)
plt.show()


Based on the previous two figures, the general understanding of the current parameters being estimated should be pretty good. When identifying normal distribution parameters with this method, deviation is usually much easier to identify than mean. It is good to note that based on the training data's parameters there might be different kind of situations. In many cases, after visualizing the log-likelihood values, not much can be said about the mean parameter, even though the potential values of the deviation parameter could be seen quite clearly. In other cases, it might be hard to say anything about the goodness of any parameter values, or it might even be easy to decrease the number of parameter values tried for both mean and deviation. Sometimes, removing one bad parameter value might lead to much better end results. Of course, when using more data, the results will be better. However, errors are always possible, and too much conclusions shouldn't be done based on these visualizations.  

####Normalization of Log-likelihood Values

The next step in the process is the normalization of the log-likelihood values. First, each value is multiplied with -0.5 and then exponentiated with `numpy`'s `exp`-function. The previous nested loops are now replaced with list comprehension. The inner loop for each subset has to be before the two outer loops for all the parameters, and the square brackets have to be used between the loops, so that the operations are done correctly to each element of the matrix. The exponentiated values are also stored to `values_exp` for later use.

In [ ]:
values = [[[np.exp(-0.5*values[i][j][k]) for k in range(n_subsets)] for j in range(n_means)] for i in range(n_devs)]
values_exp = values

As mentioned before, each position in the matrix `values` contains a list of subsets. Now, one value from each list (parameter combination) is selected and they are summed up. This is done for all subsets so that each dataset is considered. The computation is done using `np.sum` with `axis=(0,1)` as a second parameter. This means that the sum is first computed along the `i`-axis with index 0 and then along the `j`-axis with index 1. After that, `sums` contain `n_subsets` sum values where one value for each parameter combination is selected and they are summed up.

In [ ]:
sums = np.sum(values, axis=(0,1))

After that, each value in `values` is divided by the sum of its' corresponding subsets to get the likelihood values for each parameter combination to fit the training data. Basically, one dataset is selected for each parameter combination, and the likelihood value for each dataset is computed. This is done for all datasets. The calculation is done using list comprehension as before.

In [ ]:
values = [[[values[i][j][k] / sums[k] for k in range(n_subsets)] for j in range(n_means)] for i in range(n_devs)]

Now, after computing the normalized likelihood values, it is possible to compute the mean value of them for all parameter combinations. This is done with a simple list comprehension.

In [ ]:
means_nor = [[np.mean(values[i][j]) for j in range(n_means)] for i in range(n_devs)]

Next, the normalized likelihood values (red) and their means (blue) for each parameter combination are plotted. The plotting is done similar to the plotting of the log-likelihood values, with the three different angles of view of the 3D figure. The size of the mean values is scaled using `s=means_nor[i][j]*200` to get the size of plotted points with higher likelihood values bigger. That makes the figure easier to evaluate.

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=3, subplot_kw={'projection': '3d'}, figsize=(21, 6))

for ax in axs:
  ax.set_xlabel("Deviation")
  ax.set_ylabel("Mean")
  ax.set_zlabel("Likelihood")
  for i in range(n_devs):
    for j in range(n_means):
      ax.scatter3D(devs[i],means[j],values[i][j], c='red', s=3)
      ax.scatter3D(devs[i],means[j],means_nor[i][j], c='blue', s=means_nor[i][j]*200)

axs[0].view_init(elev=10, azim=-45)
axs[1].view_init(elev=0, azim=-90)
axs[2].view_init(elev=0, azim=0)

axs[1].set_yticklabels([])
axs[2].set_xticklabels([])


plt.subplots_adjust(wspace=0)
#fig.suptitle("Normalized Likelihood Values and Average Values Over All Evaluations", fontsize=15)
plt.show()

Based on the 3D figure it may be difficult to see the best parameter combination and its' likelihood value. Thus, it is now computed and printed to the screen. First, the maximum value is stored to `maxval` with `np.max`. Then, the index of the maximum value in the `means_nor` matrix is computed with `np.where` function, which input parameter is the variable in which a value is searched for, continued with `==`-sign and the value which is searched for. `np.where` doesn't work for normal Python matrix, so `means_nor` has to be modified to numpy-array with `np.array`. The `ind` variable's first item is the array of the row index, and the second item the array of the column index of the maximum value in the matrix. The maximum deviation and mean are choosed from `devs` and `means` based on the correct indexes, which must be modified from arrays of one value to integers. The best fitting parameters and the likelihood of the combination rounded to 3 decimals are printed as f-string, which makes the inclusion of the variables very easy with curly brackets.  

In [ ]:
maxval = np.max(means_nor)
ind = np.where(np.array(means_nor)==maxval)
ind_d = ind[0][0]
ind_m = ind[1][0]
max_d = devs[ind_d]
max_m = means[ind_m]
print(f"\n\nThe best fitting parameter combination is mean {max_m} and dev {max_d}, with likelihood value of {round(maxval,3)}.")

It is good to note that the likelihood value of the best combination might be surprisingly small. That is, because the number of combinations is quite big, and there could be a few combinations with almost similar likelihoods. Then, it might be not that easy to know the exact best combination, and sometimes the method does not find the best combination. Of course, the results are better if bigger datasets are used or the number or parameter combinations tried can be reduced.  

####Evaluaton of Likelihood Distributions of Mean and Deviation Parameters Alone

It might also be interesting to compute the likelihood values for each mean parameter and deviation parameter alone, and not fot their combinations. The exponentiated values are stored to `values_exp`, which is now used to compute the sum of the exponentiated values for each mean parameter and each deviation parameter with `np.sum` like before. The sum of all values is also computed for the likelihood calculation.

In [ ]:
mean_vals = np.sum(values_exp, axis=(0,2))
dev_vals = np.sum(values_exp, axis=(1,2))
sum_all = np.sum(values_exp)

Then, each value in `mean_vals` and `dev_vals` is divided by the sum of all values using list comprehension to get likelihood values for all parameters.

In [ ]:
mean_vals = [val / sum_all for val in mean_vals]
dev_vals = [val / sum_all for val in dev_vals]

The likelihood distribution values for both parameters are then plotted as subplots like before. Grids are added with `ax.grid()` to make the figures clearer.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(18, 5))

fig.suptitle("Likelihood Distribution of Mean and Deviation Parameters Alone", fontsize=15)

ax[0].plot(means, mean_vals, 'ro')
ax[0].set_xlabel("Mean Parameter")
ax[0].set_ylabel("Likelihood")
ax[0].grid()

ax[1].plot(devs, dev_vals, 'ro')
ax[1].set_xlabel("Deviation Parameter")
ax[1].set_ylabel("Likelihood")
ax[1].grid()

plt.show()

Based on the likelihood values, something more could be said about the parameters. For example, it could be easier to say which of the parameters is most probably correctly identified. It could also be seen that the likelihoods split into multiple different parameter values quite evenly. The best parameter combination based on normalized likelihood doesn't always mean that the likelihood values of these parameters alone are the best.   

